# 11-3 집합과 시퀀스

In [ ]:
# IMDB 영화 리뷰 데이터 준비

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  61.4M      0  0:00:01  0:00:01 --:--:-- 61.5M


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
for category in ('neg', 'pos'):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val',
    batch_size = batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size = batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:
  print('inputs.shape:', inputs.shape)
  print('inputs.dtype:', inputs.dtype)
  print('targets.shape:', targets.shape)
  print('targets.dtype:', targets.dtype)
  print('inputs[0]:', inputs[0])
  print('targets[0]:', targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Dan Katzir has produced a wonderful film that takes us on a roller-coaster ride through a real romance set in the troubles surrounding modern Israel.<br /><br />For anyone who's ever been in love, the film brings back the uncertainties, the insecurities and heartache that make love so bitter-sweet. The atmosphere of fear and isolation that came with the difficult times in Israel at that time just serve to intensify the feeling. Instantly, you are drawn in to Dan's plight, and you can't fail to be deeply moved.<br /><br />You can't write drama and passion like this - the contrast between the realities of Dan's desperate, snatched relationship with Iris, and the realities of a state in turmoil make this eminently watchable. If you have an ounce of passion, and have ever been in love, see this film.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dty

In [ ]:
from keras.layers import TextVectorization
# One-hot 인코딩 방식의 BoW 방식
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode='multi_hot',
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


In [ ]:
# 변환 데이터 확인
for inputs, targets in binary_1gram_train_ds:
  print('inputs.shape:', inputs.shape)
  print('inputs.dtype:', inputs.dtype)
  print('targets.shape:', targets.shape)
  print('targets.dtype:', targets.dtype)
  print('inputs[0]:', inputs[0])
  print('targets[0]:', targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
# 모델 생성
from tensorflow import keras
from keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation='relu')(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation='sigmoid')(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_1gram_keras',
                                    save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model('binary_1gram_keras')
print(f'테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}')

Epoch 1/10
625/625 [==============================] - 12s 10ms/step - loss: 0.4328 - accuracy: 0.8146 - val_loss: 0.3046 - val_accuracy: 0.8720
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2893 - accuracy: 0.8942 - val_loss: 0.2864 - val_accuracy: 0.8862
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2537 - accuracy: 0.9107 - val_loss: 0.2895 - val_accuracy: 0.8872
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2356 - accuracy: 0.9190 - val_loss: 0.3082 - val_accuracy: 0.8810
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2219 - accuracy: 0.9237 - val_loss: 0.3158 - val_accuracy: 0.8824
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2206 - accuracy: 0.9273 - val_loss: 0.3264 - val_accuracy: 0.8774
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2070 - accuracy: 0.9314 - val_loss: 0.3369 - val_accuracy: 0.8808
Epoc

In [ ]:
# 바이그램 반환 텍스트 벡터화 층 만들기
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode='multi_hot',
)

# 모델 훈련 및 테스트
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_2gram_keras',
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model('binary_2gram_keras')
print(f'테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}')

Epoch 1/10
625/625 [==============================] - 8s 11ms/step - loss: 0.3888 - accuracy: 0.8403 - val_loss: 0.2657 - val_accuracy: 0.8970
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2473 - accuracy: 0.9100 - val_loss: 0.2576 - val_accuracy: 0.9038
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2160 - accuracy: 0.9269 - val_loss: 0.2728 - val_accuracy: 0.9022
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1901 - accuracy: 0.9375 - val_loss: 0.2870 - val_accuracy: 0.9038
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1830 - accuracy: 0.9419 - val_loss: 0.3044 - val_accuracy: 0.8996
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1727 - accuracy: 0.9481 - val_loss: 0.3323 - val_accuracy: 0.8958
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1630 - accuracy: 0.9503 - val_loss: 0.3364 - val_accuracy: 0.8966
Epoch

In [ ]:
# 단어 빈도-역문서 빈도(Term Frequency-Inverse Document Frequency) 정규화
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode='tf_idf',)

# 모델 훈련 및 테스트
text_vectorization.adapt(text_only_train_ds)
tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
callbacks = [
    keras.callbacks.ModelCheckpoint('tfidf_2gram_keras',
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model('tfidf_2gram_keras')
print(f'테스트 정확도: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}')

## 11.3.3 시퀀스 모델 방식

In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_1 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                5128448   
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm_keras', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10,
          callbacks=callbacks)
model = keras.models.load_model('one_hot_bidir_lstm_keras')
print(f'테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}')

Epoch 1/10
625/625 [==============================] - 164s 253ms/step - loss: 0.5538 - accuracy: 0.7181 - val_loss: 0.3529 - val_accuracy: 0.8688
Epoch 2/10
625/625 [==============================] - 164s 262ms/step - loss: 0.3663 - accuracy: 0.8643 - val_loss: 0.2982 - val_accuracy: 0.8860
Epoch 3/10
625/625 [==============================] - 155s 247ms/step - loss: 0.2986 - accuracy: 0.8949 - val_loss: 0.5752 - val_accuracy: 0.8274
Epoch 4/10
625/625 [==============================] - 155s 249ms/step - loss: 0.2531 - accuracy: 0.9136 - val_loss: 0.3037 - val_accuracy: 0.8842
Epoch 5/10
625/625 [==============================] - 165s 264ms/step - loss: 0.2263 - accuracy: 0.9261 - val_loss: 0.2870 - val_accuracy: 0.8930
Epoch 6/10
625/625 [==============================] - 152s 242ms/step - loss: 0.1941 - accuracy: 0.9372 - val_loss: 0.2890 - val_accuracy: 0.8908
Epoch 7/10
625/625 [==============================] - 154s 247ms/step - loss: 0.1711 - accuracy: 0.9442 - val_loss: 0.3308 -

OSError: ignored

In [ ]:
# 단어 임베딩 층을 이용한 모델
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
callbacks = [keras.callbacks.ModelCheckpoint('embeddings_bidir_lstm_keras',
                                             save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10,
          callbacks=callbacks)
model = keras.models.load_model('embeddings_bidir_lstm_keras')
print(f'테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}')

## 11.4.3 트랜스포머 인코더로 영화 리뷰 감성 분류

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncode(layers.Layer):
  def __init__(self, embed_dim, desne_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiheadAttention(
        num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation='relu'),
         layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(porj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        'embed_dim': self.embed_dim,
        'num_heads': self.num_heads,
        'dense_dim': self.dense_dim,
    })
    return config

##11.5.3 트랜스포머 seq2seq

In [4]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-11-02 02:14:35--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.101.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.009s  

2023-11-02 02:14:35 (290 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [5]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [6]:
import random
print(random.choice(text_pairs))

('And what happened after that?', '[start] ¿Y qué pasó después? [end]')


In [7]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [9]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


# 인코더

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation='relu'),
         layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        'embed_dim': self.embed_dim,
        'num_heads': self.num_heads,
        'dense_dim': self.dense_dim,
    })
    return config

# 위치 임베딩

In [14]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(
        input_dim=input_dim, output_dim=output_dim)
    self.position_embeddings = layers.Embedding(
        input_dim=sequence_length, output_dim=output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions


# 디코더

In [22]:
from tensorflow.keras import layers

class TransformerDecoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention_1 = layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embed_dim
    )
    self.attention_2 = layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embed_dim
    )
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation='relu'),
         layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
    self.layernorm_3 = layers.LayerNormalization()
    self.supports_masking = True

  def get_config(self):
    config = super.get_config()
    config.update({
        'embed_dim': self.embed_dim,
        'num_heads': self.num_heads,
        'dense_dim': self.dense_dim,
    })
    return config

  # 커절 마스킹
  def get_casual_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast(i >= j, dtype='int32')
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1),
        tf.constant([1, 1], dtype=tf.int32)], axis=0)
    return tf.tile(mask, mult)

  def call(self, inputs, encoder_outputs, mask=None):
    causal_mask = self.get_casual_attention_mask(inputs)
    if mask is not None:
      padding_mask = tf.cast(
          mask[:, tf.newaxis, :], dtype='int32')
      padding_mask = tf.minimum(padding_mask, causal_mask)
    attention_output_1 = self.attention_1(
        query=inputs,
        value=inputs,
        key=inputs,
        attention_mask=causal_mask)
    attention_output_1 = self.layernorm_1(inputs + attention_output_1)
    attention_output_2 = self.attention_2(
        query=inputs,
        value=inputs,
        key=inputs,
        attention_mask=causal_mask,)
    attention_output_2 = self.layernorm_2(
        attention_output_1 + attention_output_2)
    proj_output = self.dense_proj(attention_output_2)
    return self.layernorm_3(attention_output_2 + proj_output)

In [28]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype='int64', name='english')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype='int64', name='spanish')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size, activation='softmax')(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
transformer.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30


ValueError: ignored